# IPyGPULogger Demo

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [3]:
def consume_gpu(n): return torch.ones((n, n)).cuda()
def consume_cpu(n): return np.ones((n, n))

In [4]:
from ipygpulogger import IPyGPULogger
il = IPyGPULogger().start()

RAM: Consumed Peaked  Used Total | Exec time 0.000s
CPU:        0      0     2157 MB |
GPU:        0      0     2156 MB |


In [ ]:
a = consume_cpu(2**13)

RAM: Consumed Peaked  Used Total | Exec time 0.133s
CPU:      512      0     2670 MB |
GPU:        0      0     2156 MB |


In [ ]:
a1 = consume_cpu(2**12)
a = consume_cpu(2**12)
del a1

RAM: Consumed Peaked  Used Total | Exec time 0.090s
CPU:      128    128     2286 MB |
GPU:        0      0     2156 MB |


In [ ]:
b = consume_gpu(1000)

RAM: Consumed Peaked  Used Total | Exec time 0.003s
CPU:        0      0     2290 MB |
GPU:        4      0     2160 MB |


In [ ]:
data = il.data
data
data.cpu_mem_used

IPyGPULoggerData(cpu_mem_used_delta=0.0019779205322265625, cpu_mem_peaked=0.06481266021728516, cpu_mem_used=2290, gpu_mem_used_delta=4, gpu_mem_peaked=0, gpu_mem_used=2160, time_delta=0.002718687057495117)

2290

RAM: Consumed Peaked  Used Total | Exec time 0.035s
CPU:        0      0     2290 MB |
GPU:        0      0     2160 MB |


In [ ]:
il.stop()

In [ ]:
# there should be no log appearing, after the logger has been stopped
a = consume_cpu(1000)

In [ ]:
%%javascript # prevent committing an unsaved notebook
_=IPython.notebook.save_notebook()